# August 29: ABA dataset: MAX ROIs: create ROI-masks and trial level responses

In [1]:
import os 
import sys
from os.path import join as pjoin
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle, random
import csv

from nltools import mask
from nilearn import image, masking

# plotting
import matplotlib.pyplot as plt
plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr #CITE ITS PAPER IN YOUR MANUSCRIPT

# main dirs
proj_dir = pjoin(os.environ['HOME'], 'network_dynamics')
results_dir = f"{proj_dir}/results"
month_dir = f"{proj_dir}/nb/aug22"

# folders
sys.path.insert(0, proj_dir)
import helpers.dataset_utils as dataset_utils

/home/govindas/venvs/expln-ai3.9/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


## subject-wise masks
some voxels are removed due to excess motion in each individual

In [2]:
main_mask_path = (
    f"/home/govindas/vscode-BSWIFT-mnt/ABA/dataset/preproc/masksAndCensors"
)

# set_name = f"max"
# final_mask_file = f"/home/govindas/parcellations/MAX_85_ROI_masks/MAX_ROIs_final_gm_85.nii.gz"
set_name = f"mashid"
final_mask_file = f"{proj_dir}/data/rois/mashid/final_mask_cortical+subcortical.nii.gz"

for subj in tqdm(os.listdir(main_mask_path)):
    mask_path = (
        f"{main_mask_path}/{subj}"
    )

    mask_goodVoxels_file = f"{mask_path}/mask_{set_name}_goodVoxels.nii.gz"
    # if os.path.exists(mask_goodVoxels_file): continue
    mask_goodVoxels = image.math_img(
        f'img1 * img2 * img3',
        img1=f"{mask_path}/MNI152_T1_2mm_brain_GM_02182017_goodVoxels.nii.gz",
        img2=f"{mask_path}/commonVoxelsMask.nii.gz",
        img3=f"{final_mask_file}"
    )    
    mask_goodVoxels.to_filename(mask_goodVoxels_file)

    # print(f"{subj} : {len(np.unique(mask_goodVoxels.get_fdata()))}")

  0%|          | 0/91 [00:00<?, ?it/s]

## trial level responses
- responses of the play periods aligned to the end of play period. 
- each block/trial has time range $[-11.25, 6.25]$ seconds where the block/trial ends at $0.0$ seconds.
- two TRs at the boundary of the block/trial were added extra for GLM's computational purposes. So the actual duration of the block/trial is 13TRs.

In [3]:
# collect the list of subject names
pd.DataFrame(
    np.array(list(
        map(lambda x: x[-3:], os.listdir(main_mask_path))
    ))
).to_csv(
    f"{proj_dir}/data/aba/scripts/subj_names.txt",
    header=None, 
    index=None
)

In [4]:
# run 3dDeconvolve
# bash runDeconvolve.sh 

In [15]:
main_data_dir = (
    f"/home/govindas/network_dynamics/data/aba"
    f"/data/ROI_trialwise_13TR_4plays_offset_reducedRuns/unassumed/rois-{set_name}"
)

# remove empty subj folders
for subj in os.listdir(main_data_dir):
    if len(os.listdir(f"{main_data_dir}/{subj}")) == 0:
        os.system(f"rm -rf {main_data_dir}/{subj}")
    if set_name == 'mashid' and subj == 'ABA615':
        os.system(f"rm -rf {main_data_dir}/{subj}")
        # this subj has one ROI less, better to not include this subj in analysis

# set_name = 'max'

class ARGS(): pass
args = ARGS()

args.TRIAL_LEN = 13
args.LABEL_NAMES = ['PLAY_highT', 'PLAY_lowT', 'PLAY_highR', 'PLAY_lowR']

args.SEED = 74
np.random.seed(args.SEED)

args.subjects = os.listdir(main_data_dir)
random.Random(args.SEED).shuffle(args.subjects)

'''
exploratory dataset
'''
args.explor_subjects = args.subjects[ : round(0.25 * len(args.subjects))]

In [16]:
X, raw_cols, used_cols = dataset_utils.get_aba_trial_level_responses(
    args, 
    main_data_dir, 
    args.explor_subjects
)

100%|██████████| 22/22 [00:37<00:00,  1.71s/it]


In [17]:
with open(f"{proj_dir}/data/aba/data/exploratory_data/exploratory_data_rois-{set_name}.pkl", "wb") as f:
    pickle.dump(X, f)

In [18]:
for label in X.keys():
    print(f"{label}-------")
    for x in X[label]:
        print(x.shape)

0-------
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(30, 13, 45)
(19, 13, 45)
(32, 13, 45)
(32, 13, 45)
(28, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(24, 13, 45)
(26, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(24, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(21, 13, 45)
(32, 13, 45)
1-------
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(31, 13, 45)
(20, 13, 45)
(32, 13, 45)
(32, 13, 45)
(31, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(24, 13, 45)
(28, 13, 45)
(31, 13, 45)
(32, 13, 45)
(32, 13, 45)
(24, 13, 45)
(32, 13, 45)
(31, 13, 45)
(32, 13, 45)
(23, 13, 45)
(32, 13, 45)
2-------
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(30, 13, 45)
(17, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(24, 13, 45)
(28, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(24, 13, 45)
(32, 13, 45)
(31, 13, 45)
(32, 13, 45)
(22, 13, 45)
(32, 13, 45)
3-------
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(31, 13, 45)
(18, 13, 45)
(32, 13, 45)
(32, 13, 45)
(32, 13, 45)
(3

In [12]:
args.explor_subjects[3]

'ABA615'